In [1]:
library(devtools)
library("fgsea")
library(devtools)
library(tidyverse)
library(Matrix)

Warning message:
“package ‘devtools’ was built under R version 4.3.1”
Loading required package: usethis

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2          ✔ readr     2.1.4     
✔ forcats   1.0.0.9000     ✔ stringr   1.5.0     
✔ ggplot2   3.4.3          ✔ tibble    3.2.1     
✔ lubridate 1.9.2          ✔ tidyr     1.3.0     
✔ purrr     1.0.2          
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




In [2]:
read_table_run_FGSEA <- function(de_seq2_input, location, functional_annotations){
    
    input_file_name <- here(location, de_seq2_input)
    sb_denovo <- read.table(input_file_name,
                          header = TRUE)
    
    sb_denovo.selected <- sb_denovo  %>% 
        dplyr::select(gene_name, stat)

    ranks <- deframe(sb_denovo.selected)
    
    fgseaRes <- fgsea(pathways=functional_annotations, stats=ranks, 
                  nperm=10000,
                 minSize = 10, maxSize = 600)
    
    fgseaResTidy <- fgseaRes %>%
        as_tibble()
        
    
    fgseaResTidy$celltype <- de_seq2_input

    return(fgseaResTidy)
}

In [5]:
# pathways <- read_delim("/home/jpm73279/genome_downloads/Sbicolor.v5.1.prelim.annot/Sb.functional_transcripts.txt", delim = "\t" ,
#                       col_names = c("geneID", "term", "GOID"))
# pathway_format <- pathways  %>% 
#     dplyr::select(geneID, term)  %>% 
#     distinct()  %>% 
#     group_by(term)  %>% 
#     mutate(COUNTER = 1:n()) %>%
#     ungroup() %>% 
#     group_by(term)  %>% 
#     summarise(funcion = paste(geneID, collapse=","))  %>% 
#     rename(V2 = funcion)  %>% 
#     rename(V1 = term) 
    

Rows: 74877 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): geneID, term, GOID

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [7]:
# sb_functional_annotation <- convertTerms(as.data.frame(pathway_format))


In [3]:
uf_func_genes <- read.table("/home/jpm73279/genome_downloads/Ufusca/v1.1/annotation/Ufusca.GO.gmt", sep = "\t")
#sb_functional_annotation <- convertTerms(sb_func_genes)

In [9]:
convert_to_named_list <- function(df) {
  # Split the V2 column by comma and use V1 as the names
  named_list <- lapply(df$V2, function(x) unlist(strsplit(x, ",")))
  names(named_list) <- df$V1
  return(named_list)
}

# Assuming your input dataframe is named df
uf_functional_annotation <- convert_to_named_list(uf_func_genes)

In [6]:
uf_de_seq2 <- c(
"Uf.de_novo.bundle_sheath.upregulated_genes.deseq2_output.tsv",
"Uf.de_novo.companion_cells_sieve_elements.upregulated_genes.deseq2_output.tsv",
"Uf.de_novo.epidermis.upregulated_genes.deseq2_output.tsv",
"Uf.de_novo.mesophyll.upregulated_genes.deseq2_output.tsv"
)

In [17]:
library("here")
working_location <- "/scratch/jpm73279/comparative_single_cell/06.de_novo_id_markers/02.de_novo_markers/Uf"
all_gathered_cell_type_FGSEA <- lapply(uf_de_seq2, read_table_run_FGSEA, location = working_location, functional_annotations = uf_functional_annotation)




Warning message in fgsea(pathways = functional_annotations, stats = ranks, nperm = 10000, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“All values in the stats vector are greater than zero and scoreType is "std", maybe you should switch to scoreType = "pos".”
Warning message in fgsea(pathways = functional_annotations, stats = ranks, nperm = 10000, :
“You are trying to run fgseaSimple. It is recommended to use fgseaMultilevel. To run fgseaMultilevel, you need to remove the nperm argument in the fgsea function call.”
Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“All values in the stats vector are greater than zero and scoreType is "std", maybe you should switch to scoreType = "pos".”
Warning message in fgsea(pathways = functional

In [18]:
all_gathered_cell_type_FGSEA

pathway,pval,padj,ES,NES,nMoreExtreme,size,leadingEdge,celltype
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<list>,<chr>
zinc_ion_binding,0.7751354606,0.841575643,0.2301743,0.8211473,7724,43,"Urofu.1G113700, Urofu.6G116300, Urofu.3G097600, Urofu.6G048300, Urofu.4G102400, Urofu.9G121500, Urofu.7G183000, Urofu.2G368500, Urofu.2G312000, Urofu.9G332200, Urofu.1G371600, Urofu.1G346000, Urofu.8G144900, Urofu.3G279900, Urofu.2G205500, Urofu.2G448800, Urofu.1G256700, Urofu.4G091000, Urofu.3G001000, Urofu.3G318200, Urofu.1G414000, Urofu.1G324300, Urofu.3G274200, Urofu.1G368700",Uf.de_novo.bundle_sheath.upregulated_genes.deseq2_output.tsv
protein_binding,0.6133386661,0.706268767,0.2378072,0.9553416,6133,154,"Urofu.1G125700, Urofu.3G463700, Urofu.1G375700, Urofu.5G087400, Urofu.5G079800, Urofu.1G113700, Urofu.1G083600, Urofu.9G506300, Urofu.5G037200, Urofu.9G525600, Urofu.5G194600, Urofu.9G097000, Urofu.5G072200, Urofu.9G062800, Urofu.9G533700, Urofu.5G137000, Urofu.7G302800, Urofu.3G274300, Urofu.1G393000, Urofu.9G121500, Urofu.4G224000, Urofu.3G032100, Urofu.1G076500, Urofu.1G328200, Urofu.1G383000, Urofu.5G122600, Urofu.5G207500, Urofu.4G166100, Urofu.7G183000, Urofu.9G583500, Urofu.2G367800, Urofu.2G312000, Urofu.9G576300, Urofu.7G091600, Urofu.9G332200, Urofu.1G214900, Urofu.9G082700, Urofu.8G116000, Urofu.1G364400, Urofu.2G032400, Urofu.1G371600, Urofu.7G033900, Urofu.1G119500, Urofu.2G046000, Urofu.6G116200, Urofu.2G006000, Urofu.8G007600, Urofu.1G346000, Urofu.6G139500, Urofu.9G292600, Urofu.5G455800, Urofu.3G279900, Urofu.9G527000, Urofu.9G166300, Urofu.4G088900, Urofu.2G205500, Urofu.7G249500, Urofu.9G445200, Urofu.6G136100, Urofu.3G315000, Urofu.2G448800",Uf.de_novo.bundle_sheath.upregulated_genes.deseq2_output.tsv
oxidoreductase_activity,0.0587176153,0.415245203,0.3524389,1.3161106,586,65,"Urofu.9G071000, Urofu.9G062600, Urofu.2G035100, Urofu.7G136300, Urofu.2G041200, Urofu.9G217100, Urofu.6G116300, Urofu.6G048300, Urofu.3G291100, Urofu.5G465000, Urofu.9G235100, Urofu.5G125000, Urofu.7G329400, Urofu.5G470600, Urofu.5G094400, Urofu.5G094500, Urofu.1G076500, Urofu.4G166100, Urofu.1G391700, Urofu.7G335500, Urofu.7G142600, Urofu.5G313400, Urofu.3G439300, Urofu.1G343300, Urofu.7G332400, Urofu.5G422600, Urofu.7G142500, Urofu.2G129000, Urofu.7G329300, Urofu.7G202600, Urofu.7G164300, Urofu.4G104700, Urofu.7G149700, Urofu.5G347600, Urofu.4G091000, Urofu.7G247000, Urofu.6G075800, Urofu.7G322200, Urofu.4G245200, Urofu.3G001000, Urofu.7G155200, Urofu.3G170000",Uf.de_novo.bundle_sheath.upregulated_genes.deseq2_output.tsv
metal_ion_binding,0.4801052951,0.608133374,0.2998590,1.0107118,4741,29,"Urofu.2G313200, Urofu.9G375300, Urofu.9G588600, Urofu.2G230200, Urofu.1G076500, Urofu.3G198900, Urofu.4G166100, Urofu.9G133900, Urofu.5G433800, Urofu.7G237500, Urofu.3G204700, Urofu.2G448800, Urofu.9G174800, Urofu.9G549500, Urofu.9G425200, Urofu.2G088900, Urofu.5G468500, Urofu.9G366200, Urofu.9G572600, Urofu.9G208300, Urofu.5G355200, Urofu.6G165800, Urofu.7G009200, Urofu.1G306800",Uf.de_novo.bundle_sheath.upregulated_genes.deseq2_output.tsv
ATP_binding,0.0578942106,0.415245203,0.3177129,1.2560063,578,121,"Urofu.9G489800, Urofu.1G399100, Urofu.5G338300, Urofu.9G503900, Urofu.3G463700, Urofu.1G375700, Urofu.9G382500, Urofu.8G225900, Urofu.9G506300, Urofu.5G155000, Urofu.2G361100, Urofu.5G194600, Urofu.5G072200, Urofu.5G075500, Urofu.3G044000, Urofu.5G137000, Urofu.8G028600, Urofu.1G198500, Urofu.1G108500, Urofu.2G178700, Urofu.1G094500, Urofu.3G346100, Urofu.3G102100, Urofu.4G224000, Urofu.7G233700, Urofu.5G065100, Urofu.3G211600, Urofu.9G133900, Urofu.2G219100, Urofu.7G281000, Urofu.4G020600, Urofu.4G108100, Urofu.3G358800, Urofu.3G159600, Urofu.9G048600, Urofu.2G346300, Urofu.2G058400, Urofu.2G030900, Urofu.3G443500, Urofu.2G128600, Urofu.7G263400, Urofu.4G016800, Urofu.7G335600, Urofu.9G436900, Urofu.2G141500, Urofu.7G312800, Urofu.5G254400, Urofu.9G107900, Urofu.5G455800, Urofu.5G031900, Urofu.5G003000, Urofu.9G079300, Urofu.3G447100, Urofu.9G445200, 

In [19]:
uf_cell_type_FGSEA <- bind_rows(all_gathered_cell_type_FGSEA)

In [20]:
fitlered_save <- uf_cell_type_FGSEA  %>% 
    group_by(celltype)  %>% 
    mutate_at(c('pval'), as.numeric)  %>% 
    dplyr::filter(pval < .05) %>% 
    dplyr::arrange(pval, descending = TRUE)  %>% 
    dplyr::arrange(celltype)  %>% 
    mutate(celltype = str_replace(celltype, "Uf.de_novo.", ""))  %>% 
    mutate(celltype = str_replace(celltype, ".upregulated_genes.deseq2_output.tsv", ""))  %>% 
    dplyr::select(-leadingEdge)

#write_delim(fitlered_save, "Sb.FGSEA_results.tsv", delim = "\t")

In [14]:
unique(fitlered_save$celltype)

character(0)

In [21]:
fitlered_save %>%
  group_by(celltype) %>%
  arrange(padj) %>%
  slice_head(n = 10)

pathway,pval,padj,ES,NES,nMoreExtreme,size,celltype
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
carbohydrate_metabolic_process,0.0001003915,0.003814878,0.5133943,1.807735,0,39,bundle_sheath
catalytic_activity,0.0037007401,0.068743442,0.4176881,1.545596,36,59,bundle_sheath
"photosynthesis,_light_harvesting",0.0054271139,0.068743442,0.6152125,1.732036,49,12,bundle_sheath
transmembrane_transport,0.0167008197,0.335050404,0.3814852,1.626341,162,40,companion_cells_sieve_elements
calcium_ion_binding,0.0248627704,0.335050404,0.4419681,1.624278,230,21,companion_cells_sieve_elements
nucleic_acid_binding,0.0267229255,0.335050404,-0.2930111,-1.658551,18,21,companion_cells_sieve_elements
protein-disulfide_reductase_activity,0.0291348178,0.335050404,0.4629629,1.602870,262,17,companion_cells_sieve_elements
carbohydrate_metabolic_process,0.0427368421,0.389647059,0.3906225,1.532452,405,27,companion_cells_sieve_elements
ATP_binding,0.0000999900,0.001924808,0.3604250,1.483959,0,409,epidermis


In [22]:
write_delim(fitlered_save, "/home/jpm73279/Mendieta_et_al_comparative_single_cell/metrics/supplament/Uf.FGSEA_results.tsv", delim = "\t")